In [1]:
import pickle
import operator
import jieba.analyse
import jieba

readFile = pickle.load( open('feed.pickle', 'rb'))
feeds = []
[ feeds.append(mess) for messages in readFile for mess in messages]


def most_frequent_reactor(feeds):
  users = {} 
  for i in range(0, len(feeds)):
    if 'reactions' in feeds[i]:
      for u in feeds[i]['reactions']['data']:
        if u['name'] in users:
          users[u['name']] += 1
        else:
          users[u['name']] = 1
  sorted_users = sorted(users.items(), key=operator.itemgetter(1), reverse=True)
  print (sorted_users[:20])

def most_frequent_reply(feeds):
  users = {}
  for i in range(0, len(feeds)):
    if 'comments' in feeds[i]:
      for u in feeds[i]['comments']['data']:
        user = u['from']['name']
        if user in users:
          users[user] += 1
        else:
          users[user] = 1
  sorted_users = sorted(users.items(), key=operator.itemgetter(1), reverse=True)
  print (sorted_users[:20])

def remove_prefix(message):
  if '靠北清大' in message:
    try:
      message = message.split('\n', 1)[1] 
    except:
      print (message)
  if '投稿日期' in message:
    message = message.split('投稿日期')[0]
  message=message.split('Submitted')[0]
  return message

def most_frequent_words(feeds, date):
  words = {} 
  for i in range(0, len(feeds)):
    if date in feeds[i]['created_time']:
        if 'message' in feeds[i]:
          feed = remove_prefix(feeds[i]['message'])
          keywords = jieba.analyse.extract_tags(feed, topK=20)
          for keyword in keywords:
            if keyword in words :
              words[keyword] += 1
            else:
              words[keyword] = 1
  sorted_words = sorted(words.items(), key=operator.itemgetter(1), reverse=True)
  return sorted_words 



用年份和月份來分類

In [2]:
def month2string(year, month):
    if len(str(month)) > 1: 
        return str(year) + '-' + str(month) 
    else: 
        return str(year) + '-' + '0'+str(month)
    


def get_monthly_keywords():
    # started year 
    year = 2012
    month = 1
    keywords = {}
    while year < 2018:
        month = 1
        for i in range(0, 12):
            date = month2string(year, month)
            keywords[date] = most_frequent_words(feeds, date)
            month += 1
        year += 1
    return keywords

keywords = get_monthly_keywords()
    

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/5h/jb3nrc7x2ps9z0689jszdd6w0000gn/T/jieba.cache
Loading model cost 1.913 seconds.
Prefix dict has been built succesfully.


無限期抵制靠北清大
#靠北清大40655
#靠北清大51554


Use cross validation to get the dummy keywords

In [3]:
def get_dummy_keywords(keywords):
    dummy_keywords = {}
    for key in keywords:
        for word in keywords[key][:30]:
            if word[0] in dummy_keywords:
                dummy_keywords[word[0]] += 1
            else:
                dummy_keywords[word[0]] = 1
    return sorted(dummy_keywords.items(), key=operator.itemgetter(1), reverse=True)

dummy_keywords = get_dummy_keywords(keywords)

In [4]:
feeds_filtered = []
dummy_keywords_list = [dummy[0] for dummy in dummy_keywords[:49]]


for feed in feeds:
    if 'message' in feed:
        feed_without_prefix = remove_prefix(feed['message'])
        # filter dummy_keywords
        keywords = jieba.analyse.extract_tags(feed_without_prefix, topK=20)
        keywords_filtered = [item for item in keywords if item not in dummy_keywords_list]
        
        obj_filtered = {
            'id': feed['id'],
            'message': feed_without_prefix,
            'keywords': keywords_filtered
        }
        feeds_filtered.append(obj_filtered)
        

#靠北清大51554
#靠北清大40655
無限期抵制靠北清大


In [5]:
def outputFeedsAsTxt():
    with open('./Label/feeds_200.txt','w') as f:
        count = 1
        for feed in feeds_filtered[:200]:
            f.write(str(count)+'\n')
            f.write('----')
            f.write(feed['message'])
            f.write('----\n')
            count += 1

print (len(feeds_filtered))

50781


In [6]:
# Using word2vec to get the relationship between words,
# Then using the vector between each words to classify keywords
import jieba

# jieba custom setting.
jieba.set_dictionary('jieba_dict/dict.txt.big')

# 將處室等斷詞加入dict裡面，確保後面處理文章可以斷出處室的詞
white_list = [
    '駐警隊', '學務處', '教務處'
]
for word in white_list:
    jieba.suggest_freq(word, True)
# load stopwords set
stopwordset = set()
with open('jieba_dict/stopwords.txt','r',encoding='utf-8') as sw:
    for line in sw:
         stopwordset.add(line.strip('\n'))
# breakline is also a stopword
stopwordset.add('\n')

Building prefix dict from /Users/caimingxun/NTHU/facebook-analytic/COWBEI_NTHU/jieba_dict/dict.txt.big ...
Dumping model to file cache /var/folders/5h/jb3nrc7x2ps9z0689jszdd6w0000gn/T/jieba.u78bc6de52fa0cfc9fdcb66692fb36d6f.cache
Loading model cost 2.848 seconds.
Prefix dict has been built succesfully.


In [7]:
feed_seg = open('./feed_seg.txt', 'w')
for feed in feeds_filtered:
    words = jieba.cut(feed['message'])
    for word in words:
        if word not in stopwordset:
            feed_seg.write(word +' ')

In [8]:
# word2vec
from gensim.models import word2vec
sentences = word2vec.Text8Corpus("./feed_seg.txt")
model = word2vec.Word2Vec(sentences, sg=1, size=250, window=3, min_count=1, negative=5)

# Save our model.
model.save("med250.model.bin")




In [9]:
tags = {'校隊':'課指組', '監視器':['駐警隊', '宿舍']}
inversed_model = {}

def generateModel(current_tag, inversed_model):
    for i in model.most_similar(current_tag,topn = 50):
        if i[0] not in inversed_model:
            inversed_model[i[0]] = []
            inversed_model[i[0]].append([current_tag, i[1]])
        else:
            inversed_model[i[0]].append([current_tag, i[1]])
    return inversed_model

#inversed_model = generateModel('校隊', inversed_model)
#inversed_model = generateModel('監視器', inversed_model)

print (model.wv['校隊'])


[ 0.03105933  0.0346274   0.50119489 -0.19623989 -0.00483025  0.05601816
  0.11158393  0.42633212  0.17797063  0.16752717  0.4329578   0.16424766
 -0.22314662 -0.36178422 -0.11925727 -0.15604183  0.01374474  0.21370463
 -0.2025232  -0.13408114 -0.32158548 -0.29474378  0.1829399  -0.26752621
 -0.14576812  0.15031379  0.25054905 -0.2071425   0.02577392  0.11225999
  0.18682274 -0.34739846  0.11272028  0.18173331  0.21746819  0.16094111
 -0.26806885  0.36314765 -0.47116682 -0.31355682 -0.08085117  0.10753269
  0.1783646   0.024702    0.06987718 -0.28740937 -0.02928487  0.00379328
  0.29552558  0.28180122  0.02046245  0.24336153  0.26132613  0.04614956
 -0.38102931  0.18573917 -0.04499471 -0.34409007  0.0985811  -0.02741618
 -0.04855944  0.00540773 -0.31612906  0.2114239  -0.24561089  0.05108606
  0.08701136  0.26764381 -0.26853901 -0.21524739 -0.1626671   0.00982145
  0.15242286 -0.21332097 -0.32646018  0.08534756 -0.01793155  0.20112239
  0.00375197 -0.55123246 -0.26967859  0.32689717  0

In [10]:
import numpy as np
feature_array = []

for feed in feeds_filtered[:100]:
    words = jieba.cut(feed['message'])
    array_init = np.zeros(250)
    count = 0
    for word in words:
        try:
            array_init += model.wv[word]
            count += 1
        except:
            print (word)
    array_init /= count
    feature_array.append(array_init)



的
那個
是
誰
啊
 
~
 
不要
這
~
~














了
跟


的
阿
!
!
!
!
的






 
 
的


那個
那個
 
 


而且
你
怎麼
都
不
 
那個
的
 
被
你
我
對
你
 
不過
蠻
的
 
你們
啦


(
不要
把
我
叫
去
你
了
)


(
就算
要
你
也
嗎
=
3
=
)






要
什麼
才






因為
了


但是
的




#






 
了
又






#


我
與
你
一樣
的
，


但是
和
是
。


去
《
》
的
，


他們
的
，
是
他們
的
；


去
5
於
的
。


如何
自己
的
，


如何
。


如此
才能
和
。


，
不要
獨
任何
你
的
。






:
/
/
.
.
/
/
/
/


~


我
是
可是
，
，
喲






到底
是
在
的
?


了
別人
 
結果
也
沒
自己
的


其實
沒
甚麼
啊
?


不要
嗎
?






的
?


是
的






剛才
在
的
，
的
哦
，
在
上
是
什麼樣
的
，








就
過


你


 


才
是


?


別
這樣
的
吧


還是






～


4
/
2
在
的
～
了
～


的
在
4
/


我
去
了
～
但是
～


如果
我
可以
你
"
"
否則
我
會
去
看






，




#
只要
按


#
要


#
要


#
是
的






！
！
！






4
/
與
出來


才
的
和
這麼
，
是
。




用
的
與


將
的
與


@
 
 
 


但
因
與


讓
不少
我們
將
。




其實
，
我們
在


我們
的
在
各種
的


，
在
的


，


、


讓
的
、
更為
。




然而
這些


或許
也
、
上
的


以
更為
的


同時
也
讓




所以
我們


能
讓
大家
的


不
只是
在
上


也
能
在


的
一切
。




我們
的
在
以
與
來
。


前者
以
、
的
的
與


也
就是
上
所
的
；


而
了
以
為
來


我們
有
那些
的


並
更
多
的
可能




In [11]:
# check instance number
print(len(feature_array))

100


In [12]:
labels_array = []
with open('./Label/labels_200.txt', 'r') as f:
    for line in f:
        labels_array.append(line)

In [13]:
print (len(labels_array))

100


In [14]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(feature_array, labels_array)
GaussianNB(priors=None)

GaussianNB(priors=None)

In [37]:
test_ids = [108, 102, 113, 114, 120]


for id in test_ids:
    test = feeds[id]['message']
    words = jieba.cut(test)
    test_feature = np.zeros(250)
    count = 0
    for word in words:
        try:
            test_feature += model.wv[word]
            count += 1
        except:
            pass
    test_feature /= count
    print("文章：\n")
    print(test)
    print("結果：\n")
    print(clf.predict([test_feature])[0])
    

文章：

#靠北清大51504

工院陳同學就是在說你啦！
很多女生貼很厲害？
不過是帥了點 家裡錢多一點罷了
花花公子一個！一點內涵都沒有！

Submitted:  April 21, 2017 4:27:09 PM CST
結果：

4

文章：

#靠北清大51510

早上好天氣下午給我下大雨，
老天頗有指導教授的脾氣...

Submitted:  April 21, 2017 6:18:09 PM CST
結果：

2

文章：

#靠北清大51499

人家diss你還自己跳出來承認
在那回無腦留言
長點腦好嗎
去年就已經搞到鬼屋消失了
不要明年連逃脫都辦不成

Submitted:  April 21, 2017 3:34:06 PM CST
結果：

1

文章：

#靠北清大51498

BRAVO!
邊緣人天天靠清刷存在感，
果然是現實世界沒朋友到只能在虛擬空間取暖，
上班時間都在看籃球還可以輕鬆獲得八卦被人推推，
實在令人羨慕忌妒恨

Submitted:  April 21, 2017 3:32:21 PM CST
結果：

4

文章：

#靠北清大51492

大家好，我是清大最邊緣
剛剛上課打了瞌睡，出現的居然是周公不是佛祖
我心裡一驚，難道是我來錯地方了？ 但還是硬著頭皮跟周公下了一盤棋
最後跟今日溜馬一樣被大逆轉，心裡超不是滋味

此時周公地拿出了一幅油畫，並告訴我：「你們這周末有兩團密室逃脫，這是其中一團的場地和關卡線索，當個好同學幫助大家吧！」
http://imgur.com/vFAUavu
只可惜我邊緣，不知道怎麼跟同學開口，也不曉得這到底是哪一場，更沒有朋友陪我玩什麼密室逃脫，只好PO上來
(可以幫我QQ嗎？)

最後我問周公：「佛祖不是說好今天騎士贏了就會告訴我後續的嗎？」
周公笑曰：「你等明天公牛G3贏賽爾提克再去找祂問吧！」
然後就下課了

投稿日期：  2017年4月21日 15:08 CST
結果：

4



'2\n'

In [39]:
print (test)

#
